In [1]:
import pandas as pd
import numpy as np

In [2]:
#Read both train and test dataset
train=pd.read_csv("train_file.csv")
test=pd.read_csv("test_file.csv")

FileNotFoundError: [Errno 2] File train_file.csv does not exist: 'train_file.csv'

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
train.describe()

In [ ]:
#Checked null values
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train["Source"].unique()

In [ ]:
#Checked which source is repeated most of the time found Bloomberg is repeated most of the time so replaced all null values with Bloomberg
train["Source"].value_counts()

In [ ]:
train["Topic"].value_counts()

In [ ]:
test["Source"].value_counts()

In [ ]:
#filled null values with Bloomberg
train["Source"]=train["Source"].fillna("Bloomberg")
test["Source"]=test["Source"].fillna("Bloomberg")

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
#import nltk,re,stopwords for text preprocessing and cleaning
import nltk
import re
from nltk.corpus import stopwords



In [ ]:
#made fuction for cleaning of data 
stop=set(stopwords.words('english'))
def clean(text):
  text_token = nltk.word_tokenize(text)
  filtered_text = ' '.join([w.lower() for w in text_token if w.lower() not in stop])
  filtered_text = filtered_text.replace(r"[^a-zA-Z]+", '')
  text_only = re.sub(r'\b\d+\b', '', filtered_text)
  clean_text = text_only.replace(',', '').replace('.', '').replace(':', '')
  return clean_text

In [ ]:
#Combined Title+Source+Topic for Title
#combined Headline+Source+Topic 
#as all columns have text data so we can easily preprocess it
train['Text_Title'] = train['Title'] + ' ' + train['Source'] + ' ' + train['Topic']
test['Text_Title'] = test['Title'] + ' ' + test['Source'] + ' ' + test['Topic']

train['Text_Headline'] = train['Headline'] + ' ' + train['Source'] + ' ' + train['Topic']
test['Text_Headline'] = test['Headline'] + ' ' + test['Source'] + ' ' + test['Topic']

In [ ]:
train['Text_Title'][355]

In [ ]:
#apply clean fuction to both Text_Title & Text_Headline
train['Text_Title'] = [clean(x) for x in train['Text_Title']]
test['Text_Title'] = [clean(x) for x in test['Text_Title']]

train['Text_Headline'] = [clean(x) for x in train['Text_Headline']]
test['Text_Headline'] = [clean(x) for x in test['Text_Headline']]

In [ ]:
train['Text_Title'][355]

In [ ]:
#apply TF-idf for converting text into vectors
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
#convert text into sparse matrix
vectorizer = TfidfVectorizer(use_idf=True)

train_v_Title = vectorizer.fit_transform(train['Text_Title'])
test_v_Title = vectorizer.transform(test['Text_Title'])

vectorizer_ = TfidfVectorizer()

train_v_Headline = vectorizer_.fit_transform(train['Text_Headline'])
test_v_Headline = vectorizer_.transform(test['Text_Headline'])

In [ ]:
train_v_Title 

In [ ]:
train_v_Headline

In [ ]:
print(np.shape(train_v_Title))

In [ ]:
#apply train test split
from sklearn.model_selection import train_test_split

In [ ]:
#apply SVM model to train_v_Title and SentimentTitle
X_train, X_test, y_train, y_test = train_test_split(train_v_Title , train["SentimentTitle"], test_size=0.20, random_state=42)
from sklearn.svm import LinearSVR
model1 = LinearSVR(C=0.2)
model1.fit(X_train, y_train)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
#import Mean_absolute_error
from sklearn.metrics import mean_absolute_error

In [ ]:
#check y_pred for X_test
y_pred1 = model1.predict(X_test)
#Check Mean absolute error for predicted value and test value
mae1 = mean_absolute_error(y_pred1, y_test)
print('MAE:', 1 - mae1)

In [ ]:
#done same things for SentimentHeadline which done for SentimentTitle
X_train, X_test, y_train, y_test = train_test_split(train_v_Headline, train["SentimentHeadline"], test_size=0.20, random_state=42)

model2 = LinearSVR(C=0.1)
model2.fit(X_train, y_train)

y_pred2 = model2.predict(X_test)
mae2 = mean_absolute_error(y_pred2, y_test)
print('MAE:', 1 - mae2)

In [ ]:

print('MAE:', 1 - ((0.4 * mae1) + (0.6 * mae2)))

In [ ]:
#predict values for test_v_title & test_v_Headline and store it into Title & Headline
title = model1.predict(test_v_Title)
headline = model2.predict(test_v_Headline)

In [ ]:
title.shape

In [ ]:
headline.shape

In [ ]:
#store IDLink in test_id
test_id = test['IDLink']

In [ ]:
#create dataframe of predicted values by combining test_id,title,headline
df = pd.DataFrame()
df['IDLink'] = test_id
df['SentimentTitle'] = title
df['SentimentHeadline'] = headline
df

In [ ]:
#convert it into csv file
df.to_csv("C:\\Users\\KAMLESH\\Submission.csv" ,encoding="ISO-8859-1")